In [52]:
%matplotlib inline
import utils; reload(utils)
from random import randint
from utils import *

In [53]:
path = 'data/fishes/'
full_path = os.getcwd() + '/' + path

In [54]:
batch_size = 64

In [55]:
VERSION = 'sgd_002'

### To do list:

1. Create validation set and sample
2. Move to separate dirs for each set
3. Finetune and train
4. Submit to kaggle

In [57]:
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

# Finetune and train

In [64]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

In [65]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.
Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


### Training the model

In [66]:
#trn = get_data(path + 'train')
#val = get_data(path + 'valid')

In [67]:
#test = get_data(path+'test')

In [68]:
#save_array(path+'results/trn.dat', trn)
#save_array(path+'results/val.dat', val)

In [69]:
#save_array(path+'results/test.dat', test)

Load data from files

In [70]:
trn = load_array(path + 'results/trn.dat')

In [71]:
val = load_array(path + 'results/val.dat')

In [72]:
test = load_array(path + 'results/test.dat')

### Precompute convolutional output

We pre-compute the output

In [73]:
conv_layers, fc_layers = split_at(model, Convolution2D)

In [74]:
conv_model = Sequential(conv_layers)

In [75]:
#conv_feat = conv_model.predict(trn)
#conv_val_feat = conv_model.predict(val)

In [76]:
#conv_test_feat = conv_model.predict(test)

In [77]:
#save_array(path + 'results/conv_val_feat{}.dat'.format(VERSION), conv_val_feat)
#save_array(path + 'results/conv_feat{}.dat'.format(VERSION), conv_feat)

In [78]:
#save_array(path + 'results/conv_test_feat{}.dat'.format(VERSION), conv_test_feat)

### Load precomputed conv. output

In [79]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [80]:
conv_val_feat.shape

(563, 512, 14, 14)

# Train model


In [81]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [82]:
p = 0.6  # dropout

In [83]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [84]:
bn_model.fit(
    conv_feat,
    trn_labels,
    batch_size=batch_size,
    nb_epoch=3,
    validation_data=(conv_val_feat, val_labels))

Train on 3214 samples, validate on 563 samples
Epoch 1/3
3214/3214 [==============================] - 1s - loss: 1.5925 - acc: 0.4975 - val_loss: 0.6688 - val_acc: 0.8366
Epoch 2/3
3214/3214 [==============================] - 1s - loss: 0.6477 - acc: 0.8046 - val_loss: 0.3687 - val_acc: 0.9130
Epoch 3/3
3214/3214 [==============================] - 1s - loss: 0.3913 - acc: 0.8880 - val_loss: 0.2508 - val_acc: 0.9396


In [85]:
bn_model.optimizer.lr = 0.001

In [86]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=7, 
             validation_data=(conv_val_feat, val_labels))

Train on 3214 samples, validate on 563 samples
Epoch 1/7
3214/3214 [==============================] - 1s - loss: 0.2775 - acc: 0.9235 - val_loss: 0.2109 - val_acc: 0.9361
Epoch 2/7
3214/3214 [==============================] - 1s - loss: 0.2037 - acc: 0.9431 - val_loss: 0.1692 - val_acc: 0.9520
Epoch 3/7
3214/3214 [==============================] - 1s - loss: 0.1645 - acc: 0.9546 - val_loss: 0.1685 - val_acc: 0.9503
Epoch 4/7
3214/3214 [==============================] - 1s - loss: 0.1212 - acc: 0.9673 - val_loss: 0.1417 - val_acc: 0.9645
Epoch 5/7
3214/3214 [==============================] - 1s - loss: 0.1051 - acc: 0.9736 - val_loss: 0.1431 - val_acc: 0.9591
Epoch 6/7
3214/3214 [==============================] - 1s - loss: 0.0832 - acc: 0.9804 - val_loss: 0.1304 - val_acc: 0.9609
Epoch 7/7
3214/3214 [==============================] - 1s - loss: 0.0859 - acc: 0.9813 - val_loss: 0.1186 - val_acc: 0.9680


In [87]:
bn_model.save_weights(path+'models/conv_512_6_{}.h5'.format(VERSION))

In [88]:
bn_model.evaluate(conv_val_feat, val_labels)

563/563 [==============================] - 0s     


[0.11859356453079817, 0.96802841918294846]

In [89]:
bn_model.load_weights(path+'models/conv_512_6_{}.h5'.format(VERSION))

# Submit

In [92]:
test = load_array(path+'results/test.dat')

In [93]:
#test_batches = get_batches(path + 'test', shuffle=False)
preds = bn_model.predict(conv_test_feat, batch_size=batch_size)

In [94]:
def do_clip(arr, mx):
    return np.clip(arr, (1-mx)/7, mx)

In [95]:
subm = do_clip(preds, 0.85)
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]


In [96]:
subm_name = path+'results/subm_bb_dp_{}.gz'.format(VERSION)

In [97]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429
1,img_06893.jpg,0.628228,0.021429,0.021429,0.021429,0.049763,0.175196,0.027436,0.066550
2,img_02082.jpg,0.224164,0.021429,0.021429,0.021429,0.709265,0.021429,0.053195,0.021429
3,img_06261.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429
4,img_03628.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429


In [98]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [99]:
FileLink(subm_name)

/home/ubuntu/nbs/data/fishes/results/subm_bb_dp_sgd_002.gz